<img src="../img/crowdlearning-etic.png" alt="Logo ETIC" align="right">


<h1><font color="#004D7F" size=6>Selección de Variables</font></h1>

<br>
<br>
<br>
<br>
<div style="text-align: right">
<font color="#004D7F" size=3>Antonio Jesús Gil</font><br>
<font color="#004D7F" size=3>Fundamentos de Machine Learning</font><br>

</div>

## <font color="#004D7F">Introducción a métodos de selección de variables</font>

El objetivo de la regresión __lasso__ es obtener el subconjunto de predictores que minimiza el error de predicción para una variable de respuesta cuantitativa. Lasso hace esto al imponer una restricción en los parámetros del modelo que hace que los coeficientes de regresión para algunas variables se reduzcan a cero. Las variables con un coeficiente de regresión igual a cero después del proceso de contracción se excluyen del modelo. 

Las variables con coeficientes de regresión distintos de cero están más fuertemente asociadas con la variable _target_. Por lo tanto, cuando se realiza un modelo de regresión, puede ser útil hacer una regresión __lasso__ para predecir cuántas variables debe contener el modelo. Esto asegura que su modelo no sea demasiado complejo y evita que el modelo se ajuste demasiado, lo que puede resultar en un modelo sesgado e ineficiente.


En general, existen métodos para la contracción y la selección variables de un modelo de regresión lineal. Estas son regresión de __lasso__ y regresión __ridge__. 

La regresión __ridge__ es básicamente un modelo de regresión lineal regularizado. El parámetro `λ` es un escalar que también se debe aprender, utilizando un método llamado validación cruzada.

La única diferencia entre estos dos métodos es que el término de regularización se obtiene un valor absoluto. Pero esta diferencia tiene un gran impacto en la compensación. 

El método __lasso__ supera la desventaja de la regresión __ridge__ al no solo castigar los valores altos de los coeficientes `β`, sino que también los establece en cero si no son relevantes. Por lo tanto, puede terminar con menos funciones incluidas en el modelo de las que comenzó, lo cual es una gran ventaja.

### <font color="#004D7F">Veamos un ejemplo</font>

Sobre los precios de venta en viviendas

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib
from scipy.stats import skew
import matplotlib.pyplot as plt
from IPython.display import display, HTML
from scipy.stats.stats import pearsonr
%matplotlib inline
from subprocess import check_output

In [ ]:
train=pd.read_csv('./train.csv')
test = pd.read_csv("./test.csv")
train.shape
train.head(2)

In [ ]:
# Investigar el dataset
all_data = pd.concat((train.loc[:,'MSSubClass':'SaleCondition'],
                      test.loc[:,'MSSubClass':'SaleCondition']))
all_data.head(2)

In [ ]:
# Procesado del dataset 
# Data PreProcessing
new_price = {"price":train["SalePrice"], "log(price + 1)":np.log1p(train["SalePrice"])}
prices = pd.DataFrame(new_price)
matplotlib.rcParams['figure.figsize'] = (8.0, 5.0)
prices.hist()

In [ ]:
# Data management
#log transform the target:
train["SalePrice"] = np.log1p(train["SalePrice"])
numeric_feats = all_data.dtypes[all_data.dtypes != "object"].index
skewed_feats = train[numeric_feats].apply(lambda x: skew(x.dropna())) #compute skewness
skewed_feats = skewed_feats[skewed_feats > 0.75]
skewed_feats = skewed_feats.index
skewed_feats

In [ ]:
all_data[skewed_feats] = np.log1p(all_data[skewed_feats])
all_data = pd.get_dummies(all_data)
all_data = all_data.fillna(all_data.mean())
X_train = all_data[:train.shape[0]]
X_test = all_data[train.shape[0]:]
y = train.SalePrice

#### <font color="#004D7F">Modelling a ridge regression in Python</font>

In [ ]:
###Model Ridge regression
from sklearn.linear_model import Ridge, RidgeCV, ElasticNet, LassoCV, LassoLarsCV
from sklearn.model_selection import cross_val_score
cross_val_score(Ridge(), X_train, y, scoring="neg_mean_squared_error", cv = 5)

In [ ]:
rmse_ridge = np.sqrt(-cross_val_score(Ridge(), X_train, y, scoring="neg_mean_squared_error", cv = 5))
rmse_ridge

In [ ]:
print(rmse_ridge)

look at a chart about the coefficients in the model

In [ ]:
alphas = [0.05, 0.1, 0.3, 1, 3, 5, 10, 15, 30, 50, 75]
def rmse_cv(model):
    rmse= np.sqrt(-cross_val_score(model, X_train, y, scoring="neg_mean_squared_error", cv = 5))
    return(rmse)
cv_ridge = [rmse_cv(Ridge(alpha = alpha)).mean() 
            for alpha in alphas]
cv_ridge = pd.Series(cv_ridge, index = alphas)
cv_ridge.plot(title = "Validation")
plt.xlabel("alpha")
plt.ylabel("rmse")

In [ ]:
cv_ridge.min()

#### <font color="#004D7F">Modelling a lasso regression in Python</font>

In [ ]:
###Model Lasso regression
model_lasso = LassoCV(alphas = [1, 0.1, 0.001, 0.0005]).fit(X_train, y)
rmse_cv(model_lasso).mean()

In [ ]:
###Model Lasso regression
coef = pd.Series(model_lasso.coef_, index = X_train.columns)
coef.head()

In [ ]:
print("Lasso picked " + str(sum(coef != 0)) + " variables and eliminated the other " +  str(sum(coef == 0)) + " variables")

In [ ]:
imp_coef = pd.concat([coef.sort_values().head(10),
                     coef.sort_values().tail(10)])
matplotlib.rcParams['figure.figsize'] = (8.0, 10.0)
imp_coef.plot(kind = "barh")
plt.title("Coefficients in the Lasso Model")

In [ ]:
# Residual model
matplotlib.rcParams['figure.figsize'] = (6.0, 6.0)
preds = pd.DataFrame({"preds":model_lasso.predict(X_train), "true":y})
preds["residuals"] = preds["true"] - preds["preds"]
preds.plot(x = "preds", y = "residuals",kind = "scatter")

## <font color="#004D7F">Tests ANOVA y Mutual Information</font> 

Son métricas univariadas para problemas de clasificación

> Test paramétrico __ANOVA__, tiene como hipótesis nula el que, para cada posible valor de la clase, la media de una variable no varía. Solo captura relaciones lineales

> __MI__, Mutual Information, captura cualquier tipo de relación

Cuanto mayor es el valor de la métrica, más relacion del atributo con la clase. Un valor 0 significa independencia total entre variables.

### <font color="#004D7F"> Ranking por test ANOVA</font> 

Selecciona las K mejores variables con __SelectKBest__

In [ ]:
import sklearn.datasets
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_classif

datos = sklearn.datasets.load_breast_cancer()
X = datos.data
Y = datos.target
# Añadimos 100 variables con ruido despues de las 30 originales 
# No deberian ser seleccionadas
semilla = 0
random = np.random.RandomState(semilla)
ruido = random.normal(size=(len(X), 100))
X_ruido = np.hstack([X, ruido])
X_t, X_T, Y_t, Y_T = train_test_split(X_ruido,Y,test_size=0.2, random_state=semilla)

# Seleccionar las 10 mejores segun test ANOVA
K = 10
ranking = SelectKBest(score_func = f_classif, k=K)
ranking.fit(X_t,Y_t)
X_t_proyec = ranking.transform(X_t)

# Ver ranking visual y de variables seleccionadas 
print("Ranking: {}".format(np.argsort(ranking.scores_)[::-1][:K]))
seleccionados = ranking.get_support()
plt.matshow(seleccionados.reshape(1,-1), cmap='prism')
plt.xlabel("Indice de la variable")


### <font color="#004D7F"> Ranking por test ANOVA</font> 

Porcentaje de variables seleccionadas utilizando __SelectPercentile__

In [ ]:
from sklearn.feature_selection import SelectPercentile
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

seleccion = SelectPercentile(f_classif)
pipe = Pipeline([('seleccion', seleccion), ('lr', LogisticRegression())])

score_means = list()
score_stds = list()
porcentajes = [1,5] + list(np.linspace(10,100,10))

for p in porcentajes:
    pipe.set_params(seleccion__percentile=p)
    this_scores = cross_val_score(pipe, X, Y)
    score_means.append(this_scores.mean())
    score_stds.append(this_scores.std())
    
plt.errorbar(porcentajes, score_means, np.array(score_stds))


## <font color="#004D7F"> Evaluación de métodos de selección</font> 

Este ejemplo obtiene una evaluación de la Regresión logística utilizando tanto el método __ANOVA__ como __MI__

In [ ]:
from sklearn.feature_selection import mutual_info_classif
# Seleccionar las 10 mejores segun test ANOVA
K = 10
ranking_F = SelectKBest(score_func = f_classif, k=K)
ranking_F.fit(X_t,Y_t)
X_seleccionF = ranking_F.transform(X_t)

# Seleccionar las 10 mejores por MI
K = 10
ranking_MI = SelectKBest(score_func = mutual_info_classif, k=K)
ranking_MI.fit(X_t,Y_t)
X_seleccionMI = ranking_MI.transform(X_t) # proyectar

# crear modelos
lr = LogisticRegression(); lr_F = LogisticRegression(); lr_MI = LogisticRegression();
lr.fit(X_t,Y_t); lr_F.fit(X_seleccionF,Y_t); lr_MI.fit(X_seleccionMI, Y_t)

# Proyectar test y validar modelos
XT_F =  ranking_F.transform(X_T); XT_MI = ranking_MI.transform(X_T)
score = lr.score(X_T, Y_T)
scoreF = lr_F.score(XT_F, Y_T)
scoreMI = lr_MI.score(XT_MI, Y_T)

print('Acc sin seleccion: {:.4f}'.format(score))
print('Acc con 10 mejores ANOVA: {:.4f}'.format(scoreF))
print('Acc con 10 mejores MI: {:.4f}'.format(scoreMI))